<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GeoDataframe-/-Carto" data-toc-modified-id="GeoDataframe-/-Carto-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GeoDataframe / Carto</a></span></li></ul></div>

### GeoDataframe / Carto

![mapa](../images/geo_portada.jpg)

In [19]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd


In [20]:
load_dotenv()

True

Hay APIS para un montón de cosas, aquí os dejo GEOCODE
- https://geocode.xyz/

In [3]:
donde = "madrid"
def geocode(address):
    """
    Saca las coordenadas de una dirección que le des.
    """
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":[float(data["longt"]),float(data["latt"])]}
    except:
        return data

In [4]:
datos = requests.get(f"https://geocode.xyz/{donde}?json=1").json()
datos

{'standard': {'addresst': {},
  'city': 'Madrid',
  'prov': 'ES',
  'countryname': 'Spain',
  'postal': {},
  'confidence': '0.90'},
 'longt': '-3.67930',
 'alt': {'loc': {'longt': '-3.64539',
   'prov': 'ES',
   'city': 'Madrid',
   'countryname': 'Spain',
   'postal': '28017',
   'region': {},
   'latt': '40.42882'}},
 'elevation': {},
 'latt': '40.42955'}

In [5]:
datos["latt"]

'40.42955'

In [6]:
datos["longt"]

'-3.67930'

In [7]:
madrid = geocode(donde)

In [8]:
#longitud - latitud
madrid

{'type': 'Point', 'coordinates': [-3.6793, 40.42955]}

A veces Geocode Falla, así que plan es sacar las coordenadas de la web

In [13]:
madrid = {'type': 'Point', 'coordinates': [-3.6793, 40.42955]}

¿Queremos hacernos una base de datos con todos los restaurantes Veganos de Madrid que hay en Foursquare? 
EEEEAAAASSSSYYYY (¡¡Somos programadoras!!) 🚀🙃🐭🔥🐼🤯

Lamento deciros que lo primerito es leer la documentación de la API. 
- https://developer.foursquare.com/docs/api-reference/venues/search/

#QUEYOYAESTABAREGISTRADA así que ya tengo token ....

In [10]:
#Pero lo he guardado en un .env para que no me llegue ningún e-mail BOMBA 💌💣

In [21]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [22]:
tok1

'1UO1CCM0MHA3AK15FPNUZ5QNYLZ11P0UATF4TBSCKHHJO15B'

In [23]:
#url = "https://favqs.com/api/session"
url_query = 'https://api.foursquare.com/v2/venues/explore'
vegan = "4bf58dd8d48988d1d3941735"

In [24]:
"""
Ojo con cómo ponemos la latitud y la longitud que 

"""

parametros = {
    
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{madrid.get('coordinates')[0]},{madrid.get('coordinates')[1]}",
    "query": f"{vegan}", 
    "limit": 100    
}

In [26]:
resp = requests.get(url= url_query, params = parametros).json()
#resp

In [27]:
resp.keys()

dict_keys(['meta', 'response'])

In [28]:
data = resp.get("response")

In [35]:
for k,v in data.items():
    print("key",k, "---> value", v,"\n")

key suggestedFilters ---> value {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]} 

key suggestedRadius ---> value 30000 

key headerLocation ---> value Current map view 

key headerFullLocation ---> value Current map view 

key headerLocationGranularity ---> value unknown 

key query ---> value 4bf58dd8d48988d1d3941735 

key totalResults ---> value 12 

key suggestedBounds ---> value {'ne': {'lat': -3.1740837600355443, 'lng': 40.1503059001387}, 'sw': {'lat': -4.121438105395224, 'lng': 39.647388018330595}} 

key groups ---> value [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ebd82f75c5c279833136481', 'name': 'Come Back Disco Lounge Bar Watamu', 'contact': {}, 'location': {'lat': -3.3545077926688673, 'lng': 40.01987952122293, 'labeledLatLngs': [{'label': 'display', 'lat': -3.35450779266886

In [64]:
#El value de la key groups del diccionario anterior está en tipo lista con un solo elemento
#Por eso nos quedamos con el elemento cero 👇🏻
decode = data.get("groups")[0]

In [66]:
from pandas import json_normalize
json_normalize(decode)

,type,name,items
0,Recommended Places,recommended,"[{'reasons': {'count': 0, 'items': [{'summary'..."


In [67]:
#vamos a visualizar de forma "amigable"  las keys y los values del diccionario para encontrar los datitos
#Echad un ojo a librería pprint
"""
for k,v in decode.items():
    print("key",k, "---> value", v,"\n")
"""

'\nfor k,v in decode.items():\n    print("key",k, "---> value", v,"\n")\n'

In [68]:
#Esto ya es la lista de diccionarios de la que tenemos que extraer el nombre y las coordenadas
decode_otravez = decode.get("items")
#decode_otravez

In [69]:
"""
for k,v in decode_otravez[0].items():
    print("key",k, "---> value", v,"\n")
"""

'\nfor k,v in decode_otravez[0].items():\n    print("key",k, "---> value", v,"\n")\n'

In [70]:
from functools import reduce
import operator

In [71]:
mapa_nombre =  ["venue", "name"]
mapa_latitud = ["venue", "location", "lat"]
mapa_longitud = ["venue", "location", "lng"]

In [74]:
# Mis fuentes --stackoverflow
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [75]:
lovemosclaro = []
for dic in decode_otravez:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    lovemosclaro.append(paralista)

In [76]:
dataframe = pd.DataFrame(lovemosclaro)
dataframe.head()

,name,latitud,longitud
0,Come Back Disco Lounge Bar Watamu,-3.354508,40.019880
1,"Osteria del Chianti, Malindi",-3.219514,40.125946
2,Simba Dishes Malindi,-3.217145,40.118823
3,Driftwood Club,-3.236039,40.127446
4,Diamonds Dreams Of Africa,-3.239620,40.126691


Al final hacemos un mapita con Cartoframes (quemegusta) y os dejo la DOC para que investiguéis porque no hay tiempo pa todo.
- https://carto.com/developers/cartoframes/

In [77]:
#!pip3 install cartoframes